In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterra as rt
import numpy as np
from pathlib import Path
import contextily as ctx
from matplotlib.colors import Normalize, PowerNorm


from measure_io import load_measure
from measure_io import create_measure

# global vars
CRS = 'EPSG:3310' # this is california albers

# file paths
RAW_DATA_DIR = Path("/Users/laurenwilner/Desktop/Desktop/epidemiology_PhD/data/raw/")

In [ ]:
# functions
def load_circuit(path):
    gdf = gpd.read_file(path)
    gdf = gdf.rename(columns = lambda c: c.lower()).dissolve("circuit_na")["geometry"].reset_index().to_crs(CRS)
    return gdf

def load_all_circuits():
    # clean polylines
    # delete extra cols
    # union polylines for a given circuit 
    # join all three provider circuit files into 1
    # NOTE: using polygon files since SDGE is only polygons. Polygons are Polylines with a 18m buffer.
    
    pge = load_circuit(RAW_DATA_DIR/ "2023.07.psps_data/ICA_circuits/PGE/PGE_circuits_polygons/PGE_circuits_polygons.shp")
    sce = load_circuit(RAW_DATA_DIR/ "2023.07.psps_data/ICA_circuits/SCE/SCE_circuits_polygon/SCE_circuits_polygon.shp")
    sdge = load_circuit(RAW_DATA_DIR/ "2023.07.psps_data/ICA_circuits/SDGE_circuits/SDGE_circuits.shp")
    return pd.concat([pge,sce,sdge])
    

In [ ]:
# load and format data 

# ca census tracts
ca_ct = load_measure("ca_census_tracts").to_crs(CRS)
ca_shape = ca_ct.unary_union

# zcta boundaries
zcta_boundaries = load_measure("us_zcta_boundaries").to_crs(CRS)
print("loaded boundaries")
zcta_ca = zcta_boundaries[zcta_boundaries.intersects(ca_shape)]

# households and businesses
zcta_hh = load_measure("us_zcta_households")
print("loaded households")
zcta_cbp = load_measure("us_zcta_cbp")
print("loaded businesses")

# psps data
psps = load_measure("us_circuit_psps_by_hr")
print("loaded psps")

# ca pop
ca_pop = rt.load_raster(RAW_DATA_DIR/ "CAPOP_2020_100m_TOTAL.tif").set_no_data_value(np.nan).to_crs(CRS)
print("loaded ca pop")

# load circuits
all_circuits = load_all_circuits()
print("loaded all circuits")

In [ ]:
# DENOMINATOR (population)
# step 1: join zcta onto both pixel-level pop and all_circuits (denominator)
    # overlay pixels with zctas (using sjoin because there are so many pixels per zcta)
    # overlay circuits with zctas (using overlap since the circuit:zcta ratio is smaller)
ca_gdf = ca_pop.to_gdf()
ca_gdf = ca_gdf[ca_gdf["value"].notnull()]
ca_gdf = ca_gdf.reset_index().rename(columns = {"index":"pixel_id"})
ca_gdf = ca_gdf.sjoin(zcta_ca) # join zcta onto pop pixel 
circuits_zcta = all_circuits.overlay(zcta_ca, keep_geom_type = True) # join zcta onto circuit df using an overlay

In [ ]:
# DENOMINATOR (population)
# step 2: by zcta, overlay circuits with pop. then agg to zcta 
    # by zcta: 
        # turn raster data into a geodataframe
        # subset to only pixels with population in them
        # intersect this subset with the circuit polys 
        # proportionally split pixel pop if there is some of 2 circuits in a pixel
        # now we have the pop for each segment of each circuit so we can aggregate to circuit_name-zcta
        # end with df with zcta, geometry, pop, circuit_na

all_zcta_circuits = []
zctas = circuits_zcta["zcta"].unique()

for zcta in zctas:
    # filter df ZCTA
    circuits_z = circuits_zcta[circuits_zcta['zcta'] == zcta]
    ca_gdf_z = ca_gdf[ca_gdf['zcta'] == zcta]
    
    # overlay
    circuits = circuits_z.overlay(ca_gdf_z, keep_geom_type=True)

    # clean up
    # keep one zcta col
    circuits.rename(columns={'zcta_1': 'zcta'}, inplace=True)
    circuits.drop(columns=['zcta_2'], inplace=True, errors='ignore')
    # drop 'index_right'
    circuits.drop(columns=['index_right'], errors='ignore', inplace = True)
   
    # calculations
    circuits["intersect_area"] = circuits.area
    circuits["pixel_area"] = circuits.groupby("pixel_id")["intersect_area"].transform("sum")
    circuits["pop_weight"] = circuits["intersect_area"] / circuits["pixel_area"]
    circuits["intersect_population"] = circuits["value"] * circuits["pop_weight"]
    
    # sum up population for each circuit
    circuit_pop = circuits.groupby(["circuit_na", "zcta"])["intersect_population"].sum().reset_index()
    circuit_pop.rename(columns={'intersect_population': 'pop'}, inplace=True)
    
    # Append to list
    all_zcta_circuits.append(circuit_pop)

# concatenate all DataFrames into one
circuits_pop_zcta = pd.concat(all_zcta_circuits, ignore_index=True)

In [ ]:
# DENOMINATOR
# write out clean geoparquet
# this probably wont be used in a synth control analysis but could be 
# used if we did indiv level exp-outcome analysis.
create_measure(circuits_pop_zcta, "ca_circuits_zcta_pop")

# write out geoparquet aggregated to zcta level to use for analysis for now
zcta_pop = circuits_pop_zcta.groupby("zcta")["pop"].agg("sum").reset_index()
create_measure(zcta_pop, "ca_gridded_zcta_pop")

In [ ]:
psps

In [ ]:
# NUMERATOR
# map customers_out to zctas 
# Merge the geometry information from all_circuits onto psps
numerator_df = psps.merge(all_circuits[['circuit_na', 'geometry']], left_on='circuit_name_ica', right_on='circuit_na')
numerator_df =  gpd.GeoDataFrame(numerator_df, geometry = 'geometry')
# spatial join with a zcta gdf to find overlap
circuit_zcta = numerator_df.overlay(zcta_ca, keep_geom_type = True) # this fails and crashes my kernel... 
# keep_geom_type = True drops lines and points 
# now i have a row for every intersection

# calculate the weight for each circuit-ZCTA overlap based on the proportion of the circuit that falls within the ZCTA.
# multiply the weight by the total_customers_impacted to find the customers_impacted for each ZCTA.
circuit_zcta["intersect_area"] = circuit_zcta.area
circuit_zcta["circuit_area"] = circuit_zcta.groupby("circuit_na")["intersect_area"].transform("sum")
circuit_zcta["weight"] = circuit_zcta["intersect_area"] / circuit_zcta["circuit_area"]
circuit_zcta["total_customers_impacted"] = circuits["total_customers_impacted"] * circuit_zcta["weight"]

# sum up the customers_impacted for circuits that fall within the same zcta
customers_impacted_zcta = circuit_zcta.groupby(['zcta', 'row_start', 'row_end', 'psps_event_id', 'duration', 'outage_start', 'outage_end'])['customers_impacted'].sum().reset_index()
customers_impacted_zcta

In [ ]:
# write out parquet of numerator data
create_measure(customers_impacted_zcta, "ca_zcta_customers_impacted")

In [ ]:
ca_zcta_customers_impacted

In [ ]:
zcta_population = circuits_pop_zcta.groupby('zcta')['pop'].sum().reset_index()
zcta_geometry = circuits_zcta[['zcta', 'geometry']].drop_duplicates()
zcta_gdf = zcta_geometry.merge(zcta_population, on='zcta')
# gdf with zcta-circuit agg pop:
zcta_gdf = gpd.GeoDataFrame(zcta_gdf, geometry='geometry')

def plot_zcta_population(gdf, zcta_boundaries):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # ZCTA boundaries
    zcta_boundaries.boundary.plot(ax=ax, edgecolor='black', linewidth=.05, zorder = 1)
    
    # Normalize the column for population to improve color mapping
    norm = PowerNorm(gamma=0.5, vmin=gdf['pop'].min(), vmax=gdf['pop'].max())
    
    # Overlay population data
    gdf.plot(ax=ax, column='pop', cmap='plasma', alpha = 1, legend=True,
             legend_kwds={'label': "Total Population by ZCTA", 'orientation': "horizontal"},
             norm=norm, zorder=2)
    
    ax.set_axis_off()
    plt.show()

# Call the function with your geodataframe and the zcta boundaries
plot_zcta_population(zcta_gdf, zcta_ca)

In [ ]:
customers_impacted_zcta

In [ ]:
# Questions 
# 1. do we want a row for each circuit-zcta-hr? or each zcta-hr?
    # what is the level of agg we want?
    # ANSWER: agg to zcta-hr
# 2. how do we incorporate businesses with pixel pop?
    # do we move away from using census pop + businesses?
    # HOLD ON THIS FOR NOW, maybe incorporate later into the circuit-zcta level data
# 3. for the numerator (customers impacted), how will we get from circuit to zcta? we can either: 
    # a. proportionally split the circuit across all the zctas it is in and say that % of people are out in each zip
    # b. probabilistically determine which zcta had the outage based on which zcta has the most of the circuit in it OR has the most customers in it, etc
        # since outages are usually in clusters, this may be better? 
    # ANSWER: proportionally split the circuit between zctas it is in and the customers out get proportionally split between each of the zctas. 
# 4. union of polylines - is this ok? ANSWER: Yes.

# Notes
# 1. using polygons not polylines. these are polylines with 18m buffer. 
# 2. not all of our pop is covered by these circuits
    # ca pop covered by these circuits = 29399918
    # ca pop not covered by these circuits = 39538230
# 3. for any pixel with circuit coverage in this dataset: 
    # assuming that pixel is entirely covered by the circuits in the dataset
    # so, we are saying that on a pixel by pixel basis we do or do not have coverage
    # we can thus define the pop_weight as the relative size of each circuit segment in that pixel
# 4. there are circuits that cover no people
# 5. potential source of bias:
    # most zctas have 1000+ pixels covering them
    # ~300 have less than 1000 pixels in them
    # for now we will assume that most zctas have sufficient pixel coverage

# Next steps: 
# plot

In [ ]:
# original code for circuit overlays

# not all of our population is covered by these circuits. 
# ca pop covered by these circuits = 29399918
# circuits.loc[~circuits["pixel_id"].duplicated(), "value"].sum()
# ca pop not covered by these circuits = 39538230
# ca_gdf["value"].sum()
# circuits = circuits_zcta.overlay(ca_gdf, keep_geom_type = True)

# for any pixel with circuit coverage in this dataset: 
# assuming that pixel is entirely covered by the circuits in the dataset
# so, we are saying that on a pixel by pixel basis we do or do not have coverage
# we can thus define the pop_weight as the relative size of each circuit segment in that pixel
# note: there are also circuits that cover no people, which we already know.
# circuits["intersect_area"] = circuits.area
# circuits["pixel_area"] = circuits.groupby("pixel_id")["intersect_area"].transform("sum")
# circuits["pop_weight"] = circuits["intersect_area"]/circuits["pixel_area"]
# circuits["intersect_population"] = circuits["value"]*circuits["pop_weight"]
# circuit_pop = circuits.groupby("circuit_na")["intersect_population"].sum()

# assign makes a new col called pop that will become a col in all_circuits.
# our function takes as an arg the entire df at the point of assign (after index has been set).
# then we return circuit_pop reindexed with the index from all_circuits
# essentially giving circuit_pop the same index as all_circuits
# left merge circuit_pop (right) onto all_circuits (left). keep all things in all_circuits.
# circuit_pop = all_circuits.set_index("circuit_na").assign(pop = lambda x: circuit_pop.reindex(x.index, fill_value = 0)).reset_index()

In [ ]:
alameda_ct = ca_ct[ca_ct["geoid"].str[:5] == "06001"].to_crs(ca_pop.crs)

In [ ]:
fig,ax = plt.subplots(figsize = (10,10))
ca_pop.clip(alameda_ct).mask(alameda_ct).plot(ax=ax, vmax = 200)
alameda_ct.boundary.plot(ax=ax, color = "black", linewidth = 0.05)

In [ ]:
# potential source of bias:
    # most zctas have 1000+ pixels covering them
    # ~300 have less than 1000 pixels in them
    # for now we will assume that most zctas have sufficient pixel coverage
pixels_per_zcta = zcta_ca.area/10000
pixels_per_zcta.hist()

In [ ]:
pixels_per_zcta[pixels_per_zcta < 1000].hist()

In [ ]:
# diagnostic for circuit line bits

# future: make a diagnostics.py and put all of these in there.
def examine_multiline_circuits():
    pge_polyline = gpd.read_file(RAW_DATA_DIR/ "2023.07.psps_data/ICA_circuits/PGE/PGE_circuits_lines/PGE_circuits_lines.shp")
    mask=pge_polyline["Circuit_Na"].duplicated(keep = False)
    mask=pge_polyline["Circuit_Na"] == "NORTH DUBLIN-VINEYARD" # can change this to any circuit! 
    pge_polyline[mask].sort_values("Circuit_Na").reset_index().plot("index", alpha=0.5)
    plt.show()
    
    pge_polyline["circuit_len"] = pge_polyline.length
    pge_polyline["circuit_len_prop"] = pge_polyline["circuit_len"]/pge_polyline.groupby("Circuit_Na")["circuit_len"].transform("sum")
    print(pge_polyline.groupby("Circuit_Na")["circuit_len_prop"].max().sort_values())

examine_multiline_circuits()